In [2]:
class ResultAnalyzer:
    def __init__(self):
        pass
    
    def visualize_keyframe_distribution(self, keyframe_indices, total_frames):
        """Visualiza la distribución de keyframes a lo largo del video"""
        plt.figure(figsize=(12, 4))
        plt.plot(keyframe_indices, [1] * len(keyframe_indices), 'ro', markersize=8)
        plt.xlim(0, total_frames)
        plt.ylim(0, 2)
        plt.yticks([])
        plt.xlabel('Número de Frame')
        plt.title('Distribución de Keyframes')
        plt.grid(axis='x')
        plt.tight_layout()
        
        return plt
    
    def analyze_movement_patterns(self, keyframes, landmarks_detector):
        """Analiza patrones de movimiento en los keyframes detectados"""
        if not keyframes:
            return None
            
        # Extraer landmarks de cada keyframe
        keyframe_landmarks = []
        for frame in keyframes:
            landmarks = landmarks_detector.detect_landmarks(frame)
            if landmarks is not None:
                keyframe_landmarks.append(landmarks)
        
        if len(keyframe_landmarks) < 2:
            return None
            
        # Calcular distancias entre landmarks consecutivos
        distances = []
        regions_movement = {
            'frente_cejas': [],
            'ojos': [],
            'nariz': [],
            'boca': [],
            'mejillas': []
        }
        
        for i in range(1, len(keyframe_landmarks)):
            prev_landmarks = keyframe_landmarks[i-1].reshape(-1, 2)
            curr_landmarks = keyframe_landmarks[i].reshape(-1, 2)
            
            # Calcular distancia euclidiana media entre puntos correspondientes
            dists = np.sqrt(np.sum((curr_landmarks - prev_landmarks)**2, axis=1))
            avg_dist = np.mean(dists)
            distances.append(avg_dist)
            
            # Analizar movimiento por región facial
            region_indices = {
                'frente_cejas': landmarks_detector.indices['indices_frente_cejas'],
                'ojos': landmarks_detector.indices['indices_ojos'],
                'nariz': landmarks_detector.indices['indices_nariz'],
                'boca': landmarks_detector.indices['indices_boca'],
                'mejillas': landmarks_detector.indices['indices_mejillas_pomulos']
            }
            
            # Crear mapeo de índice global a posición en landmarks
            idx_to_pos = {idx: i for i, idx in enumerate(landmarks_detector.indices_clave)}
            
            # Calcular movimiento por región
            for region_name, region_idx in region_indices.items():
                region_positions = [idx_to_pos[idx] for idx in region_idx if idx in idx_to_pos]
                if region_positions:
                    region_dists = [dists[pos] for pos in region_positions if pos < len(dists)]
                    if region_dists:
                        regions_movement[region_name].append(np.mean(region_dists))
        
        return {
            'overall_movement': distances,
            'regions_movement': regions_movement
        }
    
    def plot_movement_analysis(self, movement_data):
        """Visualiza el análisis de movimiento"""
        if movement_data is None:
            return None
            
        # Plot de movimiento general
        fig, axes = plt.subplots(2, 1, figsize=(12, 8))
        
        # Plot de movimiento general
        axes[0].plot(movement_data['overall_movement'], 'b-', marker='o')
        axes[0].set_title('Movimiento Promedio entre Keyframes Consecutivos')
        axes[0].set_xlabel('Pares de Keyframes')
        axes[0].set_ylabel('Distancia Media (píxeles)')
        axes[0].grid(True)
        
        # Plot de movimiento por región
        regions_data = movement_data['regions_movement']
        for region, values in regions_data.items():
            if values:
                axes[1].plot(values, marker='o', label=region)
                
        axes[1].set_title('Movimiento por Región Facial')
        axes[1].set_xlabel('Pares de Keyframes')
        axes[1].set_ylabel('Distancia Media (píxeles)')
        axes[1].legend()
        axes[1].grid(True)
        
        plt.tight_layout()
        return fig